In [1]:
%load_ext autoreload
%autoreload 2
import os

os.environ["WANDB_SILENT"] = "true"

In [2]:
import argparse
import os
import statistics
from pathlib import Path

import numpy as np
import pandas as pd
import torch
import yaml
from torch.optim import Adam
from torch.utils.data import DataLoader, Dataset, TensorDataset
from tqdm import tqdm
from transformers import AutoModel, AutoTokenizer

import wandb
from src import BertClassifier
from src import datasets as data_utils
from src import influence, train_utils, utils
from src.datasets import create_loo_dataset, create_test_sst2, create_train_sst2

device = utils.get_device()

config = utils.load_config(
    "model_params/bert_classifier.yaml", epochs=5, num_training_examples=1000
)

## Create Datasets

In [3]:
USE_BERT_EMBEDDINGS = True

# Create datasets
train_dataset = create_train_sst2(
    num_samples=config["num_training_examples"],
    tokenizer_name=config["bert_model_name"],
    max_seq_len=config["max_sequence_length"],
    device=device,
    use_bert_embeddings=USE_BERT_EMBEDDINGS,
)

test_dataset = create_test_sst2(
    tokenizer_name=config["bert_model_name"],
    max_seq_len=config["max_sequence_length"],
    device=device,
    use_bert_embeddings=USE_BERT_EMBEDDINGS,
)
test_dataloader = DataLoader(test_dataset, shuffle=False, batch_size=1)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:00<00:00, 14255.58it/s]
Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertModel: ['vocab_transform.weight', 'vocab_projector.weight', 'vocab_layer_norm.weight', 'vocab_transform.bias', 'vocab_projector.bias', 'vocab_layer_norm.bias']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████

## Train model

In [4]:
full_model, original_df, test_loss, test_acc = train_utils.train_bert_model(
    train_dataset=train_dataset,
    test_dataset=test_dataset,
    config=config,
    use_bert_embeddings=USE_BERT_EMBEDDINGS,
)
test_loss, test_acc

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertModel: ['vocab_transform.weight', 'vocab_projector.weight', 'vocab_layer_norm.weight', 'vocab_transform.bias', 'vocab_projector.bias', 'vocab_layer_norm.bias']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 63/63 [00:02<00:00, 31.30batch/s]


(0.41442395658465236, 80.27522935779817)

## Adversarial Attack

In [5]:
def perturb_datapoint(dataset, data_guid, perturbation):
    """This modifies the dataset in place"""
    device = utils.get_device()
    guid, inputs, attn_mask, labels = [t[data_guid] for t in train_dataset.tensors]
    assert guid.squeeze() == data_guid

    inputs_before = inputs.detach().clone()
    inputs += perturbation.to(device)
    return inputs_before, inputs


def perform_attack(
    model,
    config,
    train_dataset,
    test_dataset,
    target_test_guid,
    target_train_guid=None,
    alpha=2e-2,
):
    infl = None
    if target_train_guid is None:
        print("---Computing Influence Function---")
        infl = influence.compute_influence(
            model,
            target_test_guid,
            param_influence=list(model.classifier.parameters()),
            train_dataset=train_dataset,
            test_dataset=test_dataset,
            use_bert_embeddings=True,
            lissa_r=2,
            lissa_depth=1,
            damping=5e-3,
            scale=100,
        )

        # Most negative influence is most helpful
        helpful_idxs = np.argsort(infl)[:10]
        target_train_guid = helpful_idxs[0]

    print("---Computing Input Influence Function---")
    input_infl = influence.compute_input_influence(
        model,
        target_test_guid,
        param_influence=list(model.classifier.parameters()),
        train_dataset=train_dataset,
        test_dataset=test_dataset,
        use_bert_embeddings=True,
        lissa_r=2,
        lissa_depth=1,
        damping=5e-3,
        scale=100,
        training_indices=[target_train_guid],
    )

    print(f"---Perturbing training guid {target_train_guid}---")
    perturb = alpha * input_infl[target_train_guid]
    perturb_datapoint(train_dataset, target_train_guid, perturb)

    print("---Retraining on perturbed data---")
    # Retrain model on perturbed dataset
    model, df, full_test_loss, full_test_acc = train_utils.train_bert_model(
        train_dataset=train_dataset,
        test_dataset=test_dataset,
        config=config,
        use_bert_embeddings=True,
    )
    df["perturbed_guid"] = target_train_guid
    return model, df, infl, input_infl

In [17]:
original_df[original_df.pred == original_df.label].sort_values('loss', ascending=False)

,test_guid,logits,pred,label,loss
497,497,"[0.0020433236, -0.011170415]",0,0,0.686562
870,870,"[-0.0993969, -0.13528356]",0,0,0.675365
271,271,"[0.0107166935, 0.050429046]",1,1,0.673488
695,695,"[-0.43976375, -0.4852701]",0,0,0.670653
230,230,"[-0.25980154, -0.21333694]",1,1,0.670185
...,...,...,...,...,...
613,613,"[-3.3422766, 3.141655]",1,1,0.001527
303,303,"[-3.5994592, 2.8959098]",1,1,0.001509
837,837,"[-3.6050532, 3.4566987]",1,1,0.000857
443,443,"[-3.6812828, 3.639783]",1,1,0.000661


In [6]:
# TEST_GUID = 716
TEST_GUID = 497

baseline_test_loss = original_df[original_df.test_guid == TEST_GUID].loss.squeeze()
baseline_test_loss

0.6865621209144592

In [19]:
# target_train_guid = 262

hist = {
    "loss_df": [],
    "influence": [],
    "input_influence": [],
}
model = full_model
for i in range(15):
    model, loss_df, infl, input_infl = perform_attack(
        model=model,
        config=config,
        train_dataset=train_dataset,
        test_dataset=test_dataset,
        target_test_guid=TEST_GUID,
        alpha=5e-1,
        # target_train_guid=target_train_guid,
    )
    loss_df["iter"] = i

    hist["loss_df"].append(loss_df)
    hist["influence"].append(infl)
    hist["input_influence"].append(input_infl)

---Computing Influence Function---
LiSSA reps: 2 and num_iterations: 1000
Recursion at depth 0: norm is 13.830604
Recursion at depth 200: norm is 310.815369
Recursion at depth 400: norm is 400.959137
Recursion at depth 600: norm is 432.403717
Recursion at depth 800: norm is 442.971222
Recursion at depth 999: norm is 445.291595
Recursion at depth 0: norm is 13.849314
Recursion at depth 200: norm is 310.803345
Recursion at depth 400: norm is 402.894745
Recursion at depth 600: norm is 432.494446
Recursion at depth 800: norm is 443.604462
Recursion at depth 999: norm is 445.130676


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:06<00:00, 149.13it/s]


---Computing Input Influence Function---
LiSSA reps: 2 and num_iterations: 1000
Recursion at depth 0: norm is 13.806417
Recursion at depth 200: norm is 313.570862
Recursion at depth 400: norm is 402.886688
Recursion at depth 600: norm is 432.520599
Recursion at depth 800: norm is 444.473999
Recursion at depth 999: norm is 444.417725
Recursion at depth 0: norm is 14.480818
Recursion at depth 200: norm is 311.267944
Recursion at depth 400: norm is 402.693848
Recursion at depth 600: norm is 434.224945
Recursion at depth 800: norm is 440.267578
Recursion at depth 999: norm is 448.659424


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:08<00:00, 112.68it/s]


---Perturbing training guid 146---
---Retraining on perturbed data---


...

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:06<00:00, 148.30it/s]


---Computing Input Influence Function---
LiSSA reps: 2 and num_iterations: 1000
Recursion at depth 0: norm is 18.051645
Recursion at depth 200: norm is 408.585968
Recursion at depth 400: norm is 525.826538
Recursion at depth 600: norm is 563.662964
Recursion at depth 800: norm is 580.642761
Recursion at depth 999: norm is 580.527527
Recursion at depth 0: norm is 18.759806
Recursion at depth 200: norm is 405.813110
Recursion at depth 400: norm is 526.589111
Recursion at depth 600: norm is 567.518860
Recursion at depth 800: norm is 574.171387
Recursion at depth 999: norm is 585.055420


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:08<00:00, 120.26it/s]


---Perturbing training guid 119---
---Retraining on perturbed data---


Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertModel: ['vocab_transform.weight', 'vocab_projector.weight', 'vocab_layer_norm.weight', 'vocab_transform.bias', 'vocab_projector.bias', 'vocab_layer_norm.bias']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 63/63 [00:02<00:00, 29.85batch/s]


In [21]:
df = pd.concat(hist['loss_df'])
df[df.test_guid == TEST_GUID]

,test_guid,logits,pred,label,loss,perturbed_guid,iter
497,497,"[-0.118081115, 0.1089515]",1,0,0.813093,146,0
497,497,"[-0.1613373, 0.15220653]",1,0,0.862158,685,1
497,497,"[-0.19281432, 0.18368623]",1,0,0.899013,920,2
497,497,"[-0.20458043, 0.19545211]",1,0,0.913035,131,3
497,497,"[-0.21231705, 0.20318845]",1,0,0.922327,867,4
497,497,"[-0.23281749, 0.2236888]",1,0,0.947227,911,5
497,497,"[-0.26563478, 0.25650555]",1,0,0.987916,183,6
497,497,"[-0.29135677, 0.2822266]",1,0,1.020512,333,7
497,497,"[-0.29223296, 0.2831043]",1,0,1.021634,843,8
497,497,"[-0.29912966, 0.29000124]",1,0,1.030484,119,9


In [ ]:
df0 = loss_hist[0]
df1 = loss_hist[1]
df2 = loss_hist[2]
df3 = loss_hist[3]
df4 = loss_hist[4]

In [ ]:
df0[df0.test_guid == TEST_GUID]

In [ ]:
df1[df1.test_guid == TEST_GUID]

In [ ]:
df2[df2.test_guid == TEST_GUID]

In [ ]:
df3[df3.test_guid == TEST_GUID]

In [ ]:
df4[df4.test_guid == TEST_GUID]

## Scratch

In [ ]:
model2, loss_df2 = perform_attack(model, config, train_dataset, test_dataset, TEST_GUID)
loss_df2[loss_df2.test_guid == TEST_GUID]

In [ ]:
model3, loss_df3 = perform_attack(
    model2, config, train_dataset, test_dataset, TEST_GUID
)
loss_df3[loss_df3.test_guid == TEST_GUID]

In [ ]:
-

## Compute Influence Function

In [ ]:
TEST_GUID = 716

infl = influence.compute_influence(
    full_model,
    TEST_GUID,
    param_influence=list(full_model.classifier.parameters()),
    train_dataset=train_dataset,
    test_dataset=test_dataset,
    use_bert_embeddings=USE_BERT_EMBEDDINGS,
    lissa_r=2,
    lissa_depth=1,
    damping=5e-3,
    scale=100,
)

In [ ]:
# Most negative influence is most helpful
helpful_idxs = np.argsort(infl)[:10]
helpful_idxs

In [ ]:
np.take(infl, helpful_idxs)

## Compute Input Influence Function

In [ ]:
input_infl = influence.compute_input_influence(
    full_model,
    TEST_GUID,
    param_influence=list(full_model.classifier.parameters()),
    train_dataset=train_dataset,
    test_dataset=test_dataset,
    use_bert_embeddings=USE_BERT_EMBEDDINGS,
    lissa_r=2,
    lissa_depth=1,
    damping=5e-3,
    scale=100,
    training_indices=helpful_idxs,
)

In [ ]:
best_idx = helpful_idxs[0]
best_idx

## Perturb the Best Idx

In [ ]:
def get_guid(dataset, data_guid):
    pass


def perturb_datapoint(dataset, data_guid, perturbation):
    """This modifies the dataset in place"""
    device = utils.get_device()
    guid, inputs, attn_mask, labels = [t[data_guid] for t in train_dataset.tensors]
    assert guid.squeeze() == data_guid

    inputs_before = inputs.detach().clone()
    inputs += perturbation.to(device)
    return inputs_before, inputs

In [ ]:
alpha = 1e-2

perturb = alpha * input_infl[best_idx]
before, after = perturb_datapoint(train_dataset, best_idx, perturb)

## Retrain model with new dataset

In [ ]:
model, df, full_test_loss, full_test_acc = train_utils.train_bert_model(
    train_dataset=train_dataset,
    test_dataset=test_dataset,
    config=config,
    use_bert_embeddings=USE_BERT_EMBEDDINGS,
)

In [ ]:
test_loss = df[df.test_guid == TEST_GUID].loss.squeeze()
test_loss

In [ ]:
df[df.test_guid == TEST_GUID]

In [ ]:
--

In [ ]:
before

In [ ]:
after

In [ ]:
guid, inputs, attn_mask, labels = [t[data_guid] for t in train_dataset.tensors]

inputs[0][0]

In [ ]:
guid, inputs, attn_mask, labels = [t[262] for t in train_dataset.tensors]

In [ ]:
inputs[0][0]

In [ ]:
# inputs[0]

In [ ]:
inputs.shape

In [ ]:
inputs += perturb.to(device)

In [ ]:
inputs

In [ ]:
input_infl[262]

In [ ]:
import pickle

with open("input_infl.pkl", "wb") as fh:
    pickle.dump(input_infl, fh)

In [ ]:
262 in helpful_idxs

In [ ]:
-

## Word2Vec

In [ ]:
from gensim.models import KeyedVectors, Word2Vec

w2v = KeyedVectors.load("word2vec/glove-twitter-100.kv")

In [ ]:
w2v["spielberg"]